In [1]:
#크롤링시 필요한 라이브러리 불러오기
from bs4 import BeautifulSoup
import requests

In [2]:
# 페이지 url 형식에 맞게 바꾸어 주는 함수 만들기
  #입력된 수를 1, 11, 21, 31 ...만들어 주는 함수
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num+1
    else:
        return num+9*(num-1)

In [3]:
# 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)
def makeUrl(search,start_pg,end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(start_page)
        print("생성url: ",url)
        return url
    else:
        urls= []
        for i in range(start_pg,end_pg+1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(page)
            urls.append(url)
        print("생성url: ",urls)
        return urls

In [4]:
# html에서 원하는 속성 추출하는 함수 만들기 (기사, 추출하려는 속성값)
def news_attrs_crawler(articles,attrs):
    attrs_content=[]
    for i in articles:
        attrs_content.append(i.attrs[attrs])
    return attrs_content


In [5]:
#뉴스기사 내용 크롤링하는 함수 만들기(각 뉴스의 url)
def news_contents_crawler(news_url):
    contents=[]
    for i in news_url:
        #각 기사 html get하기
        news = requests.get(i)
        news_html = BeautifulSoup(news.text,"html.parser")
            #기사 내용 가져오기 (p태그의 내용 모두 가져오기) 
        contents.append(news_html.find_all('p'))
    return contents

In [6]:
#html생성해서 기사크롤링하는 함수 만들기(제목,url): 3개의 값을 반환함(제목, 링크, 내용)
def articles_crawler(url):
    #html 불러오기
    original_html = requests.get(i)
    html = BeautifulSoup(original_html.text, "html.parser")
    # 검색결과
    articles = html.select("div.group_news > ul.list_news > li div.news_area > a")
    title = news_attrs_crawler(articles,'title')
    url = news_attrs_crawler(articles,'href')
    content = news_contents_crawler(url)
    return title, url, content #3개의 값을 반환

In [7]:
#####뉴스크롤링 시작#####

#검색어 입력
search = input("검색할 키워드를 입력해주세요:")

#검색 시작할 페이지 입력
page = int(input("\n크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 시작 페이지: ",page,"페이지")   
#검색 종료할 페이지 입력
page2 = int(input("\n크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 종료 페이지: ",page2,"페이지")   

# naver url 생성
url = makeUrl(search,page,page2)

#뉴스 크롤러 실행
news_titles = []
news_url =[]
news_contents =[]
for i in url:
    title, url,content = articles_crawler(url)
    news_titles.append(title)
    news_url.append(url)
    news_contents.append(content)

print("검색된 기사 갯수: 총 ",(page2+1-page)*10,'개')
print("\n[뉴스 제목]")
print(news_titles)
print("\n[뉴스 링크]")
print(news_url)
print("\n[뉴스 내용]")
print(news_contents)

검색할 키워드를 입력해주세요: 대선

크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력): 1



크롤링할 시작 페이지:  1 페이지



크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력): 10



크롤링할 종료 페이지:  10 페이지
생성url:  ['https://search.naver.com/search.naver?where=news&sm=tab_pge&query=대선&start=1', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=대선&start=11', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=대선&start=21', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=대선&start=31', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=대선&start=41', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=대선&start=51', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=대선&start=61', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=대선&start=71', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=대선&start=81', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=대선&start=91']
검색된 기사 갯수: 총  100 개

[뉴스 제목]
[['[단독] 선관위, ‘대선공약 발굴’ 지시한 산업부 1차관 수사의뢰', '[인터뷰] 안철수 "대선 전 야권통합 불가…단일화 생각 없다"', "트럼프 공화당 차기 대선 주자 지지율 47%…'압도적' 1위", '민주당, 가상자산 과세유

In [19]:
###데이터 프레임으로 만들기###
import pandas as pd

#제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist
    
#제목, 링크, 내용 담을 리스트 생성
news_titles_1, news_url_1, news_contents_1 = [],[],[]

#1차원 리스트로 만들기(내용 제외)
makeList(news_titles_1,news_titles)
makeList(news_url_1,news_url)
makeList(news_contents_1,news_contents)


#데이터 프레임 만들기
news_df = pd.DataFrame({'title':news_titles_1,'link':news_url_1,'content':news_contents_1})
news_df

,title,link,content
0,"[단독] 선관위, ‘대선공약 발굴’ 지시한 산업부 1차관 수사의뢰",https://www.chosun.com/politics/politics_gener...,[]
1,"[인터뷰] 안철수 ""대선 전 야권통합 불가…단일화 생각 없다""",http://yna.kr/AKR20211102161300001?did=1195m,"[[재난포털], [기사제보], [자동완성 기능이 켜져 있습니다.], [[송고시간],..."
2,트럼프 공화당 차기 대선 주자 지지율 47%…'압도적' 1위,http://www.newsis.com/view/?id=NISX20211103_00...,"[[2021.11.03 (수)], [[서울 15℃]], [[국제]], [트럼프 공화..."
3,"민주당, 가상자산 과세유예 검토…이재명 대선 공약 가능성도",https://biz.sbs.co.kr/article_hub/20000036861?...,[]
4,[단독]내년 대선비용 2836억원…코로나·인건비로 37% ‘껑충’,http://news.heraldcorp.com/view.php?ud=2021110...,[[[헤럴드경제=이원율 기자] 중앙선거관리위원회가 제 20대 대통령 선거 관리를 위...
...,...,...,...
95,"국민의힘 당원 투표 이틀째…안철수, 본격 대선 행보",https://news.sbs.co.kr/news/endPage.do?news_id...,"[[\n, [김수영 기자], \n, [김수영 기자], \n, [], \n, [Seo..."
96,‘현역 프리미엄' 미미…대선·지선 앞둔 강원민심 요동,http://www.kwnews.co.kr/nview.asp?aid=22111020...,"[[‘현역 프리미엄' 미미…대선·지선 앞둔 강원민심 요동], [2021-11-3 (..."
97,김재원 “부동층이 거의 없는 대선...安 3%라도 위협적”,http://www.segye.com/content/html/2021/11/03/2...,"[[ ], [입력 : 2021-11-03 10:39:21 , [수정 : 2021-1..."
98,"“내가 정권교체” 세번째 출마 나선 안철수, 대선 다자구도로",https://news.kbs.co.kr/news/view.do?ncd=531467...,"[[\n, [안다영 기자], \n, [browneyes@kbs.co.kr], \n]..."
